In [10]:
!pip install tokenizers

     |████████████████████████████████| 3.6 MB 4.0 MB/s eta 0:00:01


In [11]:
!pip install transformers

     |████████████████████████████████| 4.4 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 10.4 MB/s ta 0:00:01


In [16]:
!pip install tqdm

In [15]:
!pip install torch

     |████████████████████████████████| 133.6 MB 173 kB/s eta 0:00:013    |████▉                           | 20.1 MB 14.1 MB/s eta 0:00:09     |███████▍                        | 30.9 MB 12.0 MB/s eta 0:00:09     |█████████▊                      | 40.7 MB 13.8 MB/s eta 0:00:074.7 MB/s eta 0:00:10


In [17]:
from dask.distributed import Client
# when working with clusters, specify cluster config, n_workers and worker_size
client = Client(n_workers=4, 
                       threads_per_worker=1,
                       memory_limit=0)

/opt/anaconda3/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55330 instead
  warnings.warn(


In [14]:
import dask.dataframe as dd
import pandas as pd
import dask

In [19]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55330/status,
Dashboard: http://127.0.0.1:55330/status,Workers: 4
Total threads: 4,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55331,Workers: 4
Dashboard: http://127.0.0.1:55330/status,Total threads: 4
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:55347,Total threads: 1
Dashboard: http://127.0.0.1:55354/status,Memory: 0 B
Nanny: tcp://127.0.0.1:55336,


In [ ]:
# https://github.com/facebookresearch/fairseq/blob/main/examples/translation/README.md

In [ ]:
# https://www.kaggle.com/code/abhishek/roberta-inference-5-folds

In [1]:
import numpy as np
import pandas as pd
import os
import tokenizers
import string
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

In [2]:
from torch.utils.data import Dataset
from transformers import( DistilBertConfig,
                                           DistilBertForMaskedLM,
                                           LineByLineTextDataset, DistilBertTokenizer, 
                                           DataCollatorForLanguageModeling,
                                          Trainer, TrainingArguments, pipeline)


/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import torch


In [4]:
import transformers
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re

In [5]:
MAX_LEN = 192
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 1
# ROBERTA_PATH = "../input/roberta-base"
# TOKENIZER = tokenizers.ByteLevelBPETokenizer(
#     vocab_file=f"{ROBERTA_PATH}/vocab.json", 
#     merges_file=f"{ROBERTA_PATH}/merges.txt", 
#     lowercase=True,
#     add_prefix_space=True
# )

In [6]:
# https://huggingface.co/roberta-base
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# https://www.kaggle.com/code/arvissu/roberta-base-inference-v2-0

In [7]:
import gc
import os
import sys
import time
import pickle
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import torch
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

import warnings
warnings.simplefilter('ignore')

In [198]:
import glob
import dask.bag as bag

In [ ]:
# download the model
# download the datasets from kaggle
# dataset: https://www.kaggle.com/competitions/feedback-prize-effectiveness/data


In [39]:


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [9]:
EPOCHS = 1
FOLDS = 4
lr = 2e-5
SEED = 2018
MAX_LEN = 512
BATCH_SIZE = 8
accumulation_steps = 4
seed_everything(SEED)

In [10]:
class callback:
    def __init__(self):
        self.loss = list()
        self.model = list()
    
    def put(self, model, loss):
        self.loss.append(loss)
        self.model.append(model)

    def get_model(self):
        ind = np.argmin(self.loss)
        return self.model[ind]

    
class FeedBackModel(nn.Module):
    def __init__(self, model):
        super(FeedBackModel, self).__init__()
        self.model = model
        # self.model = AutoModel.from_pretrained(model_path)
        self.linear = nn.Linear(768, 3)

    def forward(self, ids, mask):
        x = self.model(ids, mask)[0][:, 0, :]
        pred = self.linear(x)
        return pred


class FeedBackDataset(Dataset):
    def __init__(self, data, tokenizer, is_test=False):
        self.data = data
        self.is_test = is_test
        self.tokenizer = tokenizer
        # self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        
    def __getitem__(self, idx):
        text = self.data['discourse_text'].values[idx] + ' ' + self.tokenizer.sep_token*2  + ' '  + self.data['essay'].values[idx]
        if not self.is_test:
            target_value = self.data[y_cols].values[idx]
      
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=MAX_LEN
        )['input_ids'] 
        
                  
        mask = [1]*len(inputs) + [0] * (MAX_LEN - len(inputs)) 
        mask = torch.tensor(mask, dtype=torch.long)
        
        if len(inputs) != MAX_LEN:
            inputs = inputs + [self.tokenizer.pad_token_id] * (MAX_LEN - len(inputs)) 
        ids = torch.tensor(inputs, dtype=torch.long)
        
        
        
        
        if self.is_test:
            return {
                'ids': ids,
                'mask': mask,
            }
        
        else:
            targets = torch.FloatTensor(target_value)
            return {
                'ids': ids,
                'mask': mask,
                'targets': targets
            }
        
    def __len__(self):
        return len(self.data)

In [16]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [21]:
def load_model():
    """Return the model and tokenizer"""
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

    model = RobertaForSequenceClassification.from_pretrained("roberta-base")

    # tokenizer = RobertaTokenizer.from_pretrained(model_path)
    # model = RobertaForSequenceClassification.from_pretrained(model_path, return_dict=True)

    return tokenizer, model

In [269]:

tokenizer, model = load_model()

dmodel = dask.delayed(model.cpu())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [23]:
y_cols = ['discourse_effectiveness']

In [70]:
# dask portion
root_src = "./kaggle/feedback-prize-effectiveness/"
ESSAY_ROOT_PATH = root_src
train_src = root_src + "train.csv"
ddf = dd.read_csv(train_src)


In [232]:
ddf.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [227]:
# make a df with the files
# http://sujitpal.blogspot.com/2020/06/dask-mappartitions-and-almost.html
filepaths = []
for filepath in glob.iglob(ESSAY_ROOT_PATH + "train/*.txt", recursive=True):
    filepaths.append(filepath)

In [228]:
filepath_df = (bag.from_sequence(filepaths, partition_size=100)
      .map(lambda fp: { "filepath": fp })
      .to_dataframe())

In [230]:
# def fetchEssay(essay_path):
#     """
#     Read the text file of the specific essay_id
#     """
#     # return "blahblah"
#     print(essay_path)
#     print("opening path" + essay_path)
#     essay_text = open(essay_path, 'r').read()
#     return essay_text

# def handle_row(row):
#     return "san francisco"
#     # return row

# def handle_read_partition_(part):
#     # result = part.apply(lambda row: handle_row(row), axis=1)
#     result["txt"] = part.apply(lambda row: handle_row(row), axis=1)
#     # df["txt"] = df["filepath"].apply(fetchEssay)
#     return result



In [231]:

# out_df = filepath_df.map_partitions(lambda part : handle_read_partition_(part))

# out_df.compute().head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,discourse_effectiveness_,txt
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,1,san francisco
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,1,san francisco
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,1,san francisco
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,1,san francisco
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,1,san francisco


In [123]:
# # ddf.map_partitions(lambda df: df.assign (essay = open(df.essay_id + "train/" + essay_id + ".txt", 'r').read()   ) )
# def argh(df):
#     return df.assign(essay = df['essay_id'])

# def fetchEssay_map(df):
#     return df.assign(essay_path = open_essay(df['essay_id']))

# def open_essay(essay_id):
#     full_path = ESSAY_ROOT_PATH + "train/" + essay_id + ".txt"
#     print("opening essay: " + essay_id)
#     print("opening full path: " + full_path)
#     return (full_path)
#     # return open(full_path, 'r').read()

# # ddf_out = ddf.map_partitions(fetchEssay_map, meta = ('essay', 'f8'))
# ddf_out = ddf.map_partitions(fetchEssay_map)
# # ddf_out = ddf.map_partitions(argh)

0    opening essay: foo
1    opening essay: foo
Name: essay_id, dtype: object
0    opening full path: ./kaggle/feedback-prize-eff...
1    opening full path: ./kaggle/feedback-prize-eff...
Name: essay_id, dtype: object


In [239]:
# EWWW Don't like this logic.
# should be able to do something like this row-wise, but the way this works doesn't allow for that.
#  {"Ineffective": 0, "Adequate": 1, "Effective": 2}[effectiveness]
 
def handle_partition(df):
    def handle_effectiveness(value):
        if "Ineffective" in value:
            return 0
        elif "Adequate" in value:
            return 1
        else:
            return 2
    df["discourse_effectiveness_"] = df["discourse_effectiveness"].apply(handle_effectiveness)
    return df

ddf_relabled_effectiveness = ddf.map_partitions(handle_partition) 


In [240]:

result = ddf_relabled_effectiveness.compute()

In [241]:
result.head(20)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,discourse_effectiveness_
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,1
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,1
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,1
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,1
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,1
5,36a565e45db7,007ACE74B050,"though some say that life on Mars does exist, ...",Rebuttal,Ineffective,0
6,fb65fe816ba3,007ACE74B050,"It says in paragraph 7, on April 5, 1998, Mars...",Evidence,Adequate,1
7,4e472e2584fa,007ACE74B050,Everyone who thought it was made by alieans ev...,Counterclaim,Adequate,1
8,28a94d3ee425,007ACE74B050,Though people were not satified about how the ...,Concluding Statement,Adequate,1
9,d226f06362f5,00944C693682,Limiting the usage of cars has personal and pr...,Lead,Effective,2


2

In [ ]:
# training
# https://www.kaggle.com/code/arvissu/roberta-base-training-notebook-1-epoch

In [244]:
model_list = list()
kf = StratifiedKFold(n_splits=FOLDS)

In [263]:
end_idx = len(df)

In [270]:
df = ddf_relabled_effectiveness
for i, (train_idx, valid_idx) in enumerate(kf.split(df, y=df['essay_id'])):
    print (f"i: {i} ,train_idx: {train_idx},  valid_idx: {valid_idx}" )
    print(f'fold {i+1}')
    gc.collect()
    
    cb = callback()
    train_loader = torch.utils.data.DataLoader(FeedBackDataset(df.loc[train_idx:end_idx].reset_index(drop=True), tokenizer), batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(FeedBackDataset(df.loc[valid_idx :end_idx].reset_index(drop=True), tokenizer), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    
    
    
    
    
#     net = FeedBackModel(model)
#     net.cuda()
    
#     loss_fn = torch.nn.CrossEntropyLoss()
#     optimizer = AdamW(net.parameters(), lr = lr)    
#     param_optimizer = list(net.named_parameters())
#     no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
#     optimizer_grouped_parameters = [
#         {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#         {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#     ]
#     num_train_optimization_steps = int(EPOCHS * len(train_loader) / accumulation_steps)
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.05 * num_train_optimization_steps,
#                                                 num_training_steps=num_train_optimization_steps)  # PyTorch scheduler
#     scaler = torch.cuda.amp.GradScaler()

i: 0 ,train_idx: [    3     4     5 ... 36762 36763 36764],  valid_idx: [    0     1     2 ... 36747 36760 36761]
fold 1


ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [ ]:
for i, (train_idx, valid_idx) in enumerate(kf.split(ddf_relabled_effectiveness, y=ddf_relabled_effectiveness['essay_id'])):
    print(f'fold {i+1}')
    gc.collect()
    
    cb = callback()
    train_loader = torch.utils.data.DataLoader(FeedBackDataset(df.loc[train_idx, :].reset_index(drop=True), model_path), batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(FeedBackDataset(df.loc[valid_idx, :].reset_index(drop=True), model_path), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    
    net = FeedBackModel(model)
    net.cuda()
    
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = AdamW(net.parameters(), lr = lr)    
    param_optimizer = list(net.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    num_train_optimization_steps = int(EPOCHS * len(train_loader) / accumulation_steps)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.05 * num_train_optimization_steps,
                                                num_training_steps=num_train_optimization_steps)  # PyTorch scheduler
    scaler = torch.cuda.amp.GradScaler()
    
    for epoch in range(EPOCHS):  

        start_time = time.time()
        avg_loss = 0.0
        net.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        loss_list = []
        val_loss_list = []
        
        for step, data in enumerate(tbar):

            # get the inputs
            input_ids = data['ids'].cuda()
            input_masks = data['mask'].cuda()
            targets = data['targets'].long().view(-1).cuda()
            with torch.cuda.amp.autocast():
                pred = net(input_ids,input_masks)
                loss = loss_fn(pred, targets)
                
            scaler.scale(loss).backward()
            

            if step % accumulation_steps == 0 or step == len(tbar) - 1:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()
                
            loss_list.append(loss.detach().cpu().item())
            avg_loss = np.round(np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {epoch + 1} Loss: {avg_loss} lr: {scheduler.get_last_lr()}")
        
        net.eval()
        avg_val_loss = 0.0   
        tbar_val = tqdm(val_loader, file=sys.stdout)
        for step, data in enumerate(tbar_val):

            # get the inputs
            input_ids = data['ids'].cuda()
            input_masks = data['mask'].cuda()
            targets = data['targets'].long().view(-1).cuda()
            
            pred = net(input_ids,input_masks)
            loss = loss_fn(pred, targets)
                
            val_loss_list.append(loss.detach().cpu().item())
            avg_val_loss = np.round(np.mean(val_loss_list), 4)

            tbar_val.set_description(f"Epoch {epoch + 1} Loss: {avg_val_loss}")
                    
        cb.put(net, avg_val_loss)   
        
    model_list.append(cb.get_model())

In [ ]:
# Done generating a model.
model_target = "roberta_modellist.pkl"
with open(model_target,"wb") as f:
    pickle.dump(model_list, f)


In [ ]:
# Infer
# https://www.kaggle.com/code/arvissu/roberta-base-inference-v2-0/notebook

In [9]:
NFOLDS = 5
BATCH_SIZE = 32
MAX_LEN = 512


model_list = pickle.load(open(model_target, "rb"))
test_pred = np.zeros((len(test_df), 3))

# test_df = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
test_ddf = dd.read_csv("../input/feedback-prize-effectiveness/test.csv")

NameError: name 'torch' is not defined

In [ ]:

def predict(test_df: pd.DataFrame, tokenizer, result):
    test_loader = torch.utils.data.DataLoader(FeedBackDataset(test_df, tokenizer, True), batch_size=BATCH_SIZE, shuffle=False)
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            input_ids = data['ids'].cuda()
            input_masks = data['mask'].cuda()
            pred = F.softmax(net(input_ids,input_masks))
            result.extend(pred.cpu().detach().numpy())

In [ ]:
# Mika's note: K-folds is a fairly parallel work as the each batch in folds are normalized to have data parallelism.
# Idea hers is to use dask dataframe to handle that part.

for idx in range(NFOLDS):
    print(f'start to inference fold : {idx}')
    net = model_list[idx]
    net.eval()
    net.cuda()
    result = list()
    predict(test_ddf, tokenizer, result)
    test_pred += np.array(result)/NFOLDS
    

In [ ]:
# for idx in range(NFOLDS):
#     print(f'start to inference fold : {idx}')
#     net = model_list[idx]
#     net.eval()
#     net.cuda()
#     result = list()
#     with torch.no_grad():
#         for i, data in enumerate(test_loader):
#             input_ids = data['ids'].cuda()
#             input_masks = data['mask'].cuda()
#             pred = F.softmax(net(input_ids,input_masks))
#             result.extend(pred.cpu().detach().numpy())
#     test_pred += np.array(result)/NFOLDS

In [ ]:
# this is our test set predictions
test_pred
